In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
df = df.drop(columns = ['Id', 'Alley', 'FireplaceQu', 'MiscFeature', 'Fence', 'PoolQC', 'MasVnrType'])

In [5]:
df.Utilities.fillna('AllPub', inplace = True)
df.Exterior1st.fillna('VinylSd', inplace = True)
df.Exterior2nd.fillna('VinylSd', inplace = True)
df.LotFrontage.fillna(df.LotFrontage.mean(), inplace = True)
df.MasVnrArea.fillna(0, inplace = True)
df.BsmtQual.fillna('TA', inplace = True)
df.BsmtCond.fillna('TA', inplace = True)
df.BsmtExposure.fillna('No', inplace = True)
df.BsmtFinType1.fillna('GLQ', inplace = True)
df.BsmtFinSF1.fillna(df.BsmtFinSF1.mean(), inplace = True)
df.BsmtFinType2.fillna('Unf', inplace = True)
df.BsmtFinSF2.fillna(0.0, inplace = True)
df.BsmtUnfSF.fillna(0.0, inplace = True)
df.TotalBsmtSF.fillna(df.TotalBsmtSF.mean(), inplace = True)
df.BsmtFullBath.fillna(0.0, inplace = True)
df.BsmtHalfBath.fillna(0.0, inplace = True)
df.KitchenQual.fillna('TA', inplace = True)     
df.Functional.fillna('Typ', inplace = True)
df.GarageType.fillna('Attchd', inplace = True)
df.GarageYrBlt.fillna(round(df.GarageYrBlt.mean()), inplace = True)
df.GarageFinish.fillna('Unf', inplace = True)
df.GarageCars.fillna(2.0, inplace = True)
df.GarageArea.fillna(round(df.GarageArea.mean()), inplace = True)
df.GarageQual.fillna('TA', inplace = True)
df.GarageCond.fillna('TA', inplace = True)
df.SaleType.fillna('WD', inplace = True)

In [6]:
for column in df:
    print(column, df[column].isnull().sum(), )

MSSubClass 0
MSZoning 0
LotFrontage 0
LotArea 0
Street 0
LotShape 0
LandContour 0
Utilities 0
LotConfig 0
LandSlope 0
Neighborhood 0
Condition1 0
Condition2 0
BldgType 0
HouseStyle 0
OverallQual 0
OverallCond 0
YearBuilt 0
YearRemodAdd 0
RoofStyle 0
RoofMatl 0
Exterior1st 0
Exterior2nd 0
MasVnrArea 0
ExterQual 0
ExterCond 0
Foundation 0
BsmtQual 0
BsmtCond 0
BsmtExposure 0
BsmtFinType1 0
BsmtFinSF1 0
BsmtFinType2 0
BsmtFinSF2 0
BsmtUnfSF 0
TotalBsmtSF 0
Heating 0
HeatingQC 0
CentralAir 0
Electrical 1
1stFlrSF 0
2ndFlrSF 0
LowQualFinSF 0
GrLivArea 0
BsmtFullBath 0
BsmtHalfBath 0
FullBath 0
HalfBath 0
BedroomAbvGr 0
KitchenAbvGr 0
KitchenQual 0
TotRmsAbvGrd 0
Functional 0
Fireplaces 0
GarageType 0
GarageYrBlt 0
GarageFinish 0
GarageCars 0
GarageArea 0
GarageQual 0
GarageCond 0
PavedDrive 0
WoodDeckSF 0
OpenPorchSF 0
EnclosedPorch 0
3SsnPorch 0
ScreenPorch 0
PoolArea 0
MiscVal 0
MoSold 0
YrSold 0
SaleType 0
SaleCondition 0
SalePrice 0


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [8]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   int64  
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   int64  
 5   LotShape       1460 non-null   int64  
 6   LandContour    1460 non-null   int64  
 7   Utilities      1460 non-null   int64  
 8   LotConfig      1460 non-null   int64  
 9   LandSlope      1460 non-null   int64  
 10  Neighborhood   1460 non-null   int64  
 11  Condition1     1460 non-null   int64  
 12  Condition2     1460 non-null   int64  
 13  BldgType       1460 non-null   int64  
 14  HouseStyle     1460 non-null   int64  
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [10]:
y = df.SalePrice
df = df.drop(columns = 'SalePrice', axis = 1)
X = df
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 73 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   int64  
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   int64  
 5   LotShape       1460 non-null   int64  
 6   LandContour    1460 non-null   int64  
 7   Utilities      1460 non-null   int64  
 8   LotConfig      1460 non-null   int64  
 9   LandSlope      1460 non-null   int64  
 10  Neighborhood   1460 non-null   int64  
 11  Condition1     1460 non-null   int64  
 12  Condition2     1460 non-null   int64  
 13  BldgType       1460 non-null   int64  
 14  HouseStyle     1460 non-null   int64  
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  YearBuilt      1460 non-null   int64  
 18  YearRemo

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
regr = RandomForestRegressor(n_estimators = 100, max_depth = 10, max_features = 'sqrt', random_state = 42, replace = True)
regr.fit(X,y)

TypeError: RandomForestRegressor.__init__() got an unexpected keyword argument 'replace'

In [ ]:
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df_test_id = df_test.Id

In [ ]:
df_test

In [ ]:
df_test = df_test.drop(columns = ['Id', 'Alley', 'FireplaceQu', 'MiscFeature', 'Fence', 'PoolQC', 'MasVnrType'])
df_test.Utilities.fillna('AllPub', inplace = True)
df_test.Exterior1st.fillna('VinylSd', inplace = True)
df_test.Exterior2nd.fillna('VinylSd', inplace = True)
df_test.LotFrontage.fillna(df.LotFrontage.mean(), inplace = True)
df_test.MasVnrArea.fillna(0, inplace = True)
df_test.BsmtQual.fillna('TA', inplace = True)
df_test.BsmtCond.fillna('TA', inplace = True)
df_test.BsmtExposure.fillna('No', inplace = True)
df_test.BsmtFinType1.fillna('GLQ', inplace = True)
df_test.BsmtFinSF1.fillna(df.BsmtFinSF1.mean(), inplace = True)
df_test.BsmtFinType2.fillna('Unf', inplace = True)
df_test.BsmtFinSF2.fillna(0.0, inplace = True)
df_test.BsmtUnfSF.fillna(0.0, inplace = True)
df_test.TotalBsmtSF.fillna(df.TotalBsmtSF.mean(), inplace = True)
df_test.BsmtFullBath.fillna(0.0, inplace = True)
df_test.BsmtHalfBath.fillna(0.0, inplace = True)
df_test.KitchenQual.fillna('TA', inplace = True)     
df_test.Functional.fillna('Typ', inplace = True)
df_test.GarageType.fillna('Attchd', inplace = True)
df_test.GarageYrBlt.fillna(round(df.GarageYrBlt.mean()), inplace = True)
df_test.GarageFinish.fillna('Unf', inplace = True)
df_test.GarageCars.fillna(2.0, inplace = True)
df_test.GarageArea.fillna(round(df.GarageArea.mean()), inplace = True)
df_test.GarageQual.fillna('TA', inplace = True)
df_test.GarageCond.fillna('TA', inplace = True)
df_test.SaleType.fillna('WD', inplace = True)
for col in df_test.columns:
    if df_test[col].dtype == 'object':
        df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
ans = regr.predict(df_test)

In [13]:
Submit = pd.DataFrame({'Id': df_test_id, 'SalePrice': ans})
Submit

NameError: name 'df_test' is not defined

In [ ]:
Submit.to_csv("submit1.csv", index = False) 